# Cars and tanks trial run

In [1]:
# Preference variables
LAST_TRAIN_MODE=False

In [2]:
import os
import requests

model_file = "yolo11_cnt_last.pt" if LAST_TRAIN_MODE else "yolo11_cnt_best.pt"

if not os.path.exists(model_file):
    dl_link = {
        "yolo11_cnt_best.pt": "https://drive.google.com/uc?export=download&id=1zfCXSVrKpa38N9EauT-Z18xXgFPl6kuI",
        "yolo11_cnt_last.pt": "https://drive.google.com/uc?export=download&id=1_gkYs4yPl8kTgON-uJBKfsX3bAP5cf42"
    }

    with requests.get(dl_link[model_file], stream=True) as response:
        response.raise_for_status()
        with open(model_file, "wb") as f:
            for c in response.iter_content(chunk_size=1024 * 256):
                f.write(c)
        print("Pretrained file has been downloaded successfully.")


This notebook evaluates accuracy from external source images as well as annotate the object

In [3]:
from ultralytics import YOLO
import cv2

from pathlib import Path

img_data_dir = Path("data/images/")
imgs = [str(candidate)
        for ext_pattern in (
            '*.jpg',
            '*.png',
            '*.jpeg'
        )
        for candidate in img_data_dir.glob(ext_pattern)]

if not imgs:
    raise FileNotFoundError("Detection aborted due to no images with eligible extensions (JPG or PNG format) found.")

model = YOLO(model_file)

results = model.predict(imgs, conf=0.7, imgsz=256)

out_dir = Path("data/cnt_result/")

for idx, r in enumerate(results):
    origin_name = Path(imgs[idx]).stem
    export_path = out_dir / f"{origin_name}_predict.jpg"
    cv2.imwrite(str(export_path.absolute()), r.plot())


0: 256x256 car 1.00, tank 0.00, 7.0ms
1: 256x256 car 1.00, tank 0.00, 7.0ms
2: 256x256 car 1.00, tank 0.00, 7.0ms
3: 256x256 car 1.00, tank 0.00, 7.0ms
4: 256x256 car 1.00, tank 0.00, 7.0ms
5: 256x256 car 1.00, tank 0.00, 7.0ms
6: 256x256 car 1.00, tank 0.00, 7.0ms
7: 256x256 car 1.00, tank 0.00, 7.0ms
8: 256x256 car 1.00, tank 0.00, 7.0ms
9: 256x256 car 1.00, tank 0.00, 7.0ms
10: 256x256 car 1.00, tank 0.00, 7.0ms
11: 256x256 car 1.00, tank 0.00, 7.0ms
12: 256x256 car 1.00, tank 0.00, 7.0ms
13: 256x256 car 1.00, tank 0.00, 7.0ms
14: 256x256 car 1.00, tank 0.00, 7.0ms
15: 256x256 car 1.00, tank 0.00, 7.0ms
16: 256x256 car 1.00, tank 0.00, 7.0ms
17: 256x256 car 1.00, tank 0.00, 7.0ms
18: 256x256 car 1.00, tank 0.00, 7.0ms
19: 256x256 car 1.00, tank 0.00, 7.0ms
20: 256x256 car 1.00, tank 0.00, 7.0ms
21: 256x256 car 1.00, tank 0.00, 7.0ms
22: 256x256 car 1.00, tank 0.00, 7.0ms
23: 256x256 car 1.00, tank 0.00, 7.0ms
24: 256x256 car 0.88, tank 0.12, 7.0ms
25: 256x256 car 1.00, tank 0.00, 7

In [4]:
import pandas as pd
import numpy as np

data = {
    'Probability means': [
        np.mean([float(r.probs.data[0]) for r in results[0:99]] # Car
                + [float(r.probs.data[1]) for r in results[100:200]]), # Tank
        np.mean([float(r.probs.data[1]) for r in results[0:99]] # Car (false detect as tank)
                + [float(r.probs.data[0]) for r in results[100:200]]), # Tank (false detect as car)
    ]
}

print("Probability mean of classification accuracy")
pd.DataFrame(data, index=["True", "False"])

Probability mean of classification accuracy


,Probability means
True,0.981571
False,0.018429


In [5]:
conf_matrix = {
    "car": {
        "car": sum(r.probs.top1 == 0 for r in results[0:99]),
        "tank": sum(r.probs.top1 == 1 for r in results[0:99]),
    }, "tank": {
        "car": sum(r.probs.top1 == 0 for r in results[100:200]),
        "tank": sum(r.probs.top1 == 1 for r in results[100:200]),
    }
}

print("Confusion matrix")
pd.DataFrame(conf_matrix, index=["car", "tank"])

Confusion matrix


,car,tank
car,98,2
tank,1,98
